In [69]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt
import openpyxl 
import os
from pathlib import Path

In [70]:
import xlsxwriter
import send_email

# IMPORT DATA

In [71]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,57,58,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
df.columns

Index(['Date', 'Scheduling Time', 'Offer', 'Hitpath_Offer_ID', 'DP.SV',
       'Affiliate_Id', 'DP&Pub', 'Job_Id', 'Job_Name', 'Creative_Id',
       'Creativename', 'Creative', 'Send Strategy', 'Shortcode',
       'Start_Tstamp', 'Sent', 'Segments', 'Revenue', 'Conversions',
       'Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route', 'Carrier',
       'Dataset', 'Message', 'Responder Template', 'Keyword', 'Responder',
       'Router Domain Name', 'c1', 'Responded', 'Response Rate', 'CTR',
       'Gross Profit', 'Gross Margin', 'RPU', 'Provider', 'Code_Type',
       'Revenue Source', 'Ar Day', 'Sub_Id', 'Campaign_Id', 'Roi',
       'Shortcode Name', 'Total', 'Jump Page Version', 'Offer Wall Id',
       'Offer Type', 'campaign_id', 'campaign_name', 'shortcode_DP.SV',
       'Opportunity Cost Send Strategy', 'Dataset_Agg_30D_eCPM',
       'Opportunity Cost', 'Offer Vertical', 'Offer Gap', 'Vertical Gap',
   

In [73]:
# SMS testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')
sms_test['Hitpath Offer ID'] = sms_test['Hitpath Offer ID'].astype(str)
# SMS offer sheet 
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]

#SMS Offer wall
sms_ow = infrastructure.get_smartsheet('ow_sms')
# sms_offer = sms_offer[sms_offer['Hitpath ID'].str.contains("TEST") == False]
# sms_offer = sms_offer[~sms_offer['Hitpath ID'].str.contains(r'test\d*', regex=True)]
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype(int).astype(str)
# Email offer sheet 
email_offer = infrastructure.get_smartsheet('offers_email')
# get mamba for shortcode 
mamba = infrastructure.get_mamba()
# Offer Testing Timeline 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
timeline = gc.open_by_url('https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=423634805') 
timeline_wks  = timeline[1].get_as_df()

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/Users/nathan/Documents/SMS DMA Python/SMS DataSource/infrastructure.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  snakes['Hitpath Offer ID']=snakes['Hitpath Offer ID'].astype(str).str.replace('.0', '')


In [74]:
sms_offer.columns

Index(['Hitpath Offer ID', 'Email Hitpath Offer ID', 'Scheduling Name',
       'Offer Name', 'Status', 'Status Changed Date', 'Delayed Reporting',
       'RX Rep', 'Advertiser', 'Data Partner Relationship',
       'Advertiser Offer ID', 'Direct Traffic Accepted ',
       'Additional Notes/Pre-pop Instructions', 'Vertical', 'Redirect Link',
       'Custom Creative Allowed', 'Custom Content Status',
       'Adv/Offer Abbreviation', 'Advertiser Rep', 'Scheduled Test Date',
       'Day Restrictions', 'Other Restrictions', 'Payout Change',
       'Payout Change Effective Date', '$ Payout', 'Payout Type',
       'Budget Status', 'Budget', 'Cap', 'Portal Acct Info', 'Live Pub IDs',
       'Paused Pubs', 'Special Pub Payouts', 'Pitch Approval',
       'Single/Multiple Pitch'],
      dtype='object')

In [75]:
df.columns

Index(['Date', 'Scheduling Time', 'Offer', 'Hitpath_Offer_ID', 'DP.SV',
       'Affiliate_Id', 'DP&Pub', 'Job_Id', 'Job_Name', 'Creative_Id',
       'Creativename', 'Creative', 'Send Strategy', 'Shortcode',
       'Start_Tstamp', 'Sent', 'Segments', 'Revenue', 'Conversions',
       'Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route', 'Carrier',
       'Dataset', 'Message', 'Responder Template', 'Keyword', 'Responder',
       'Router Domain Name', 'c1', 'Responded', 'Response Rate', 'CTR',
       'Gross Profit', 'Gross Margin', 'RPU', 'Provider', 'Code_Type',
       'Revenue Source', 'Ar Day', 'Sub_Id', 'Campaign_Id', 'Roi',
       'Shortcode Name', 'Total', 'Jump Page Version', 'Offer Wall Id',
       'Offer Type', 'campaign_id', 'campaign_name', 'shortcode_DP.SV',
       'Opportunity Cost Send Strategy', 'Dataset_Agg_30D_eCPM',
       'Opportunity Cost', 'Offer Vertical', 'Offer Gap', 'Vertical Gap',
   

In [76]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [77]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

In [78]:
#sms offer restrictions
sms_restrictions = sms_offer[['Hitpath Offer ID', 'Paused Pubs']].copy()
sms_live_restrictions = sms_offer[['Hitpath Offer ID', 'Live Pub IDs']].copy()

In [79]:
#rename and retype
df.rename(columns={'Hitpath_Offer_ID': 'Hitpath Offer ID'}, inplace=True)
df['Hitpath Offer ID'] = pd.to_numeric(df['Hitpath Offer ID'], errors='coerce')
df = df.dropna(subset=['Hitpath Offer ID'])
df['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype(int).astype(str)

df['Affiliate_Id'] = pd.to_numeric(df['Affiliate_Id'], errors='coerce')
df = df.dropna(subset=['Affiliate_Id'])
df['Affiliate_Id'] = df['Affiliate_Id'].astype(int).astype(str)


In [80]:
df = df[df['Send Strategy'] != 'AR']

In [81]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [82]:
#mamba schedule from today and onward
today = pd.to_datetime(dt.date.today())

mamba_future = mamba[mamba['Date'] >= today]


# Most Recent Passed Offers

In [83]:
sms_offer['Status Changed Date'] = pd.to_datetime(sms_offer['Status Changed Date'], errors='coerce')

#get the current date to calculate the last 30 days range
current_date = pd.to_datetime('now').normalize()

sms_offer_filtered = sms_offer[
    ((sms_offer['Status'] == 'Live') | (sms_offer['Status'] == 'Live - Budgeted Offer')) &
    (sms_offer['Status Changed Date'] >= current_date - pd.Timedelta(days=30))
]

sms_offer_filtered.reset_index(drop=True, inplace=True)
sms_offer_filtered = sms_offer_filtered[
    ['Hitpath Offer ID', 'Scheduling Name', 'Advertiser', '$ Payout', 'Payout Type', 'Vertical', 'Day Restrictions']
]

In [84]:
sms_ow['Scheduled Test Date'] = pd.to_datetime(sms_ow['Scheduled Test Date'], errors='coerce')

sms_ow_filtered = sms_ow[
    (sms_ow['Status'] == 'Live')  & 
    (sms_ow['Scheduled Test Date'] >= current_date - pd.Timedelta(days=30))]

sms_ow_filtered.reset_index(drop=True, inplace=True)
sms_ow_filtered = sms_ow_filtered[['Hitpath Offer ID', 'Vertical']]

In [85]:
sms_offer_filtered = pd.concat([sms_offer_filtered, sms_ow_filtered], sort=False)

In [86]:
#calculating eCPM for passed offers from PT performance
df_pt = df[df['Send Strategy'] == 'PT']
merged_df = pd.merge(sms_offer_filtered, df_pt, on='Hitpath Offer ID', how='left')

# Grouping by 'Hitpath ID' and calculating total revenue and total delivered for each
grouped_stats = merged_df.groupby('Hitpath Offer ID').agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()
passed_offers = pd.merge(sms_offer_filtered, grouped_stats, on='Hitpath Offer ID', how='left')
passed_offers['eCPM'] = passed_offers['Revenue']/passed_offers['Delivered']*1000

In [87]:
#vertical rec
df_vertical = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_vertical = df_vertical.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

In [88]:
grouped_data = df_vertical.groupby(['Vertical', 'Sub Vertical']).agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()

# eCPM for grouped data
grouped_data['eCPM'] = (grouped_data['Revenue']/grouped_data['Delivered'])*1000

# Filtering for combinations where the calculated eCPM is greater than $16
filtered_combinations = grouped_data[grouped_data['eCPM'] > 12]

# Getting all unique sub-verticals with eCPM > $16 for each vertical
sub_verticals_eCPM_over_16 = filtered_combinations.groupby('Vertical')['Sub Vertical'].unique()

# Mapping these sub-verticals back to the 'passed offers' dataframe
passed_offers['Data Vertical Recommendations'] = passed_offers['Vertical'].map(sub_verticals_eCPM_over_16)

In [89]:
passed_offers

,Hitpath Offer ID,Scheduling Name,Advertiser,$ Payout,Payout Type,Vertical,Day Restrictions,Revenue,Delivered,eCPM,Data Vertical Recommendations
0,13125,13125-Auto Warranty-Admediary-Stateside Auto W...,Admediary,$24.00,CPL,Auto Warranty,NaN,120.00,7554.0,15.885624,"[Benefits, Credit Card, Financial, Personal Lo..."
1,13399,13399-Credit Card-B2Direct-Choice Gold Card v4...,B2Direct,$6.00,CPA,Credit Card,NaN,144.00,5402.0,26.656794,"[, Credit Card, Financial, Personal Loan, Surv..."
2,13387,13387-Home Sale-Cx3-Smart Timeshare Owner [D] ...,Cx3,$31.00,CPA,Home Sale,NaN,62.00,8003.0,7.747095,[Surveys]
3,13059,13059-Credit Score-Cx3-EXCLUSIVE - 2024FreeScores,Cx3,$38.00,CPA,Credit Score,NaN,152.00,7173.0,21.190576,[Financial]
4,13475,13475-Legal-Legal Claim Assistant-LCA - Chat F...,Legal Claim Assistant,85%,RevShare,Legal,Monday - Friday,89.76,7555.0,11.880874,"[Financial, Sweepstakes]"
5,13362,13362 EXCLUSIVE - Benefit Relief - Housing Rel...,Smooth Marketing,Variable,RevShare,Resources,NaN,119.00,6215.0,19.147224,"[Benefits, Credit Card, Financial, Personal Lo..."


# Last 30 Days of Raw Data

In [90]:
#last 30 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_30 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)
df_30 = df_30[df_30['Send Strategy'] != 'AR']



<ipython-input-90-964be40463c5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
<ipython-input-90-964be40463c5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)


In [91]:
#dataset eCPM last 30 days
eCPM_df = df_30.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']

# Historical Offer Performance

In [92]:
#total historical offer performance
offer_performance = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID']).agg({'Revenue':'sum','Delivered':'sum', 'Opportunity Cost' : 'sum'}).reset_index()
offer_performance['Hitpath Offer ID'] = offer_performance['Hitpath Offer ID'].astype(int).astype(str)
offer_performance.rename(columns={'Revenue' : 'Overall Revenue',
                                 'Delivered': 'Overall Delivered'}, inplace=True)

#match shortcode_dp.sv
values_to_match = eCPM_df['shortcode_DP.SV']
offer_performance = offer_performance[offer_performance['shortcode_DP.SV'].isin(values_to_match)]
offer_performance['Overall Offer eCPM'] = offer_performance['Overall Revenue']*1000/offer_performance['Overall Delivered']

In [93]:
#last date drop

# Convert the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Group by the unique value(s) and find the index of the most recent date
idx = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID'])['Date'].idxmax()

# Use the index to select the rows with the most recent date
last_drop = df.loc[idx]
last_drop = last_drop[['Affiliate_Id','shortcode_DP.SV', 'Offer', 'Date', 'Delivered', 'Revenue']]
last_drop['Last Drop eCPM'] = last_drop['Revenue'] * 1000 / last_drop['Delivered']

# Merge the dataframes on the 'shortcode_DP.SV' and 'Offer' columns
offer_performance = pd.merge(offer_performance, last_drop, on=['shortcode_DP.SV', 'Offer'], how='left')
offer_performance = offer_performance.rename(columns={
    'Date': 'Last Drop Date',
})

offer_performance.rename(columns={'Delivered': 'Most Recent Delivered Volume', 
                                  'Revenue' : 'Most Recent Offer Revenue'}, inplace=True)

In [94]:
#check if the drop occured in the last 14 days

# Get the current date
current_date = pd.to_datetime('today')

# Calculate the difference in days between the current date and the 'Last Drop Date'
days_difference = (current_date - offer_performance['Last Drop Date']).dt.days

# Add a new column that tells if the date was within the last 14 days
offer_performance['Last 14 Days'] = days_difference <= 14

In [95]:
sorted_offer_performance = offer_performance.sort_values(by=['shortcode_DP.SV', 'Last Drop eCPM', 'Last 14 Days'], 
                                                    ascending=[True, False, True])

#merge statuses
sorted_offer_performance = sorted_offer_performance.merge(sms_offer[['Hitpath Offer ID', 'Status']], 
                                                          how='left', 
                                                          on='Hitpath Offer ID')

#select only production statuses
sorted_offer_performance = sorted_offer_performance[
    sorted_offer_performance['Status'] == 'Live']

In [96]:
#overall dataset eCPM
newdf = sorted_offer_performance.groupby('shortcode_DP.SV').agg({'Overall Revenue':'sum','Overall Delivered':'sum'}).reset_index()
newdf['Overall Dataset eCPM'] =  newdf['Overall Revenue']*1000/newdf['Overall Delivered']
newdf = newdf.drop(columns=['Overall Revenue', 'Overall Delivered'], axis=1)

In [97]:
#merge dataset eCPM
sorted_offer_performance = sorted_offer_performance.merge(newdf, on="shortcode_DP.SV", how="left")

#merge pubid restrictions
sorted_offer_performance = sorted_offer_performance.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [98]:
sms_live_restrictions['Live Pub IDs'] = sms_live_restrictions['Live Pub IDs'].apply(
    lambda x: ','.join([str(int(float(id))).strip() for id in str(x).split(',') if id.replace('.', '', 1).isdigit()]) if pd.notna(x) else x
)

# Merge the redrop_df with live_restrictions_df on 'Hitpath Offer ID'
merged_redrop_df = sorted_offer_performance.merge(sms_live_restrictions[['Hitpath Offer ID', 'Live Pub IDs']], on='Hitpath Offer ID', how='left')

# Define the filtering function based on 'Affiliate_Id'
def filter_redrop(row):
    if pd.isna(row['Live Pub IDs']):
        return True  # No restrictions, keep the recommendation
    live_pub_ids = row['Live Pub IDs'].split(',')
    current_affiliate_id = str(row['Affiliate_Id']).strip()
    return current_affiliate_id in live_pub_ids

# Apply the updated filter function to the merged dataframe
sorted_offer_performance = merged_redrop_df[merged_redrop_df.apply(filter_redrop, axis=1)]
sorted_offer_performance = sorted_offer_performance.drop(columns=['Live Pub IDs'])

In [99]:
#remove rows if aff id is in pubid restriction
sorted_offer_performance = sorted_offer_performance[~sorted_offer_performance.apply(lambda row: str(row['Affiliate_Id']) in str(row['Paused Pubs']), axis=1)]

#drop aff id and paused pub column
sorted_offer_performance = sorted_offer_performance.drop(columns=['Affiliate_Id','Paused Pubs'], axis = 1)

#drop mismatching ids
sorted_offer_performance['offer_abbreviation'] = sorted_offer_performance['Offer'].str[-3:]

# Extract the 3 letter shortcode abbreviation from the 'shortcode_DP.SV' column
sorted_offer_performance['shortcode_abbreviation'] = sorted_offer_performance['shortcode_DP.SV'].str.split('_').str[0]

# Filter rows where the abbreviations do not match
sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['offer_abbreviation'] == sorted_offer_performance['shortcode_abbreviation']]

# Drop the temporary columns
sorted_offer_performance = sorted_offer_performance.drop(columns=['offer_abbreviation', 'shortcode_abbreviation'])

In [100]:
sorted_offer_performance.loc[sorted_offer_performance['Hitpath Offer ID'] == '12342']

,shortcode_DP.SV,Offer,Hitpath Offer ID,Overall Revenue,Overall Delivered,Opportunity Cost,Overall Offer eCPM,Last Drop Date,Most Recent Delivered Volume,Most Recent Offer Revenue,Last Drop eCPM,Last 14 Days,Status,Overall Dataset eCPM
97,DSS_TLG.PL,12342 - LG PAL DSS,12342,387.83,14050.0,198.832891,27.603559,2024-06-08,4503.0,90.39,20.073284,True,Live,16.884474
765,HZB_TLG.PL,12342 - LG PAL HZB,12342,14.40,1977.0,-20.988128,7.283763,2024-05-18,1977.0,14.40,7.283763,False,Live,20.280595


In [101]:
# #within 70% of overall
# sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['Last Drop eCPM'] >= 0.7 * sorted_offer_performance['Overall Dataset eCPM']]


In [102]:
grouped = sorted_offer_performance.groupby('shortcode_DP.SV')
filtered_dfs = []

for name, group in grouped:
    # Filtering offers with Overall Offer eCPM >= Overall Dataset eCPM
    # filtered_group = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM']]
    filtered_group = group[group['Overall Offer eCPM'] >= group['Overall Dataset eCPM']]

    # If less than 7 offers, include offers with at least 90% of Overall Dataset eCPM, then 85%, and so on.
    thresholds = [0.9, 0.85, 0.8, 0.75, 0.7]
    for threshold in thresholds:
        if len(filtered_group) < 7:
#             additional_offers = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM'] * threshold]
            additional_offers = group[group['Overall Offer eCPM'] >= group['Overall Dataset eCPM'] * threshold]
            filtered_group = pd.concat([filtered_group, additional_offers]).drop_duplicates()
        else:
            break
   
    if len(filtered_group) > 7:
        filtered_group = filtered_group.sort_values(by='Overall Offer eCPM', ascending=False).head(7)

    filtered_dfs.append(filtered_group)

# Combining all filtered groups into a single DataFrame
filtered_df = pd.concat(filtered_dfs)

In [103]:
#Opp cost eCPM
filtered_df['Opportunity Cost eCPM'] = filtered_df['Opportunity Cost']*1000/filtered_df['Overall Delivered']
filtered_df = filtered_df.drop(columns=['Opportunity Cost'])

In [104]:
#Move Opporunity Cost eCPM Column
column_to_move = filtered_df.pop("Opportunity Cost eCPM")

# insert column with insert(location, column_name, column_value)

filtered_df.insert(5, "Opportunity Cost eCPM", column_to_move)

In [105]:
# BY DMA Account
filtered_df['extracted_DP.SV'] = filtered_df['shortcode_DP.SV'].str.split('_').str[-1]

#merge using 'extracted_DP.SV' and 'DP.SV'
filtered_df = pd.merge(filtered_df, pub_config[['DP.SV', 'DMA']], 
                     left_on='extracted_DP.SV', right_on='DP.SV', how='left')

#drop temp columns
filtered_df = filtered_df.drop(columns=['extracted_DP.SV', 'DP.SV'])

In [106]:
dma_separated_dfs = {}
for dma in filtered_df['DMA'].unique():
    if pd.notna(dma):  # Ensure that the 'DMA' value is not NaN
        dma_separated_dfs[dma] = filtered_df[filtered_df['DMA'] == dma]

# Create Data for Usage in Recommended Drops

In [107]:
#last 60 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_60 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

In [108]:
sms_offer.columns

Index(['Hitpath Offer ID', 'Email Hitpath Offer ID', 'Scheduling Name',
       'Offer Name', 'Status', 'Status Changed Date', 'Delayed Reporting',
       'RX Rep', 'Advertiser', 'Data Partner Relationship',
       'Advertiser Offer ID', 'Direct Traffic Accepted ',
       'Additional Notes/Pre-pop Instructions', 'Vertical', 'Redirect Link',
       'Custom Creative Allowed', 'Custom Content Status',
       'Adv/Offer Abbreviation', 'Advertiser Rep', 'Scheduled Test Date',
       'Day Restrictions', 'Other Restrictions', 'Payout Change',
       'Payout Change Effective Date', '$ Payout', 'Payout Type',
       'Budget Status', 'Budget', 'Cap', 'Portal Acct Info', 'Live Pub IDs',
       'Paused Pubs', 'Special Pub Payouts', 'Pitch Approval',
       'Single/Multiple Pitch'],
      dtype='object')

In [109]:
# match dataset vertical to dp.sv
# df_dv_merged = df_60.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_dv_merged = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')

# match offer vertical with hitpath ID
df_dvov_merged = df_dv_merged.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

#rename
df_dvov_merged.rename(columns={'Vertical': 'Offer Vertical', 'Sub Vertical' : 'Data Vertical'}, inplace=True)


In [110]:
#only approved offers
df_dvov_merged = df_dvov_merged[df_dvov_merged['Status'] == 'Live']

In [111]:
#active pubs
df_dvov_merged = df_dvov_merged[df_dvov_merged['INTERNAL STATUS'] == 'ACTIVE']

In [112]:
#delivered > 0
df_dvov_merged = df_dvov_merged[df_dvov_merged['Delivered'] > 0]


In [113]:
#remove lead group specific offers
a = ['12384', '12385', '12386']
df_dvov_merged = df_dvov_merged[~df_dvov_merged['Hitpath Offer ID'].isin(a)]


# Offer Vertical OCE By Data Vertical

In [114]:
df_dvov_merged = df_dvov_merged.loc[:,~df_dvov_merged.columns.duplicated()].copy()


In [115]:
#find data


data_offer_vert_OCE = df_dvov_merged.groupby(['shortcode_DP.SV', 'Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()




In [116]:
df_dvov_merged

,Date,Scheduling Time,Offer,Hitpath Offer ID,DP.SV,Affiliate_Id,DP&Pub,Job_Id,Job_Name,Creative_Id,Creativename,Creative,Send Strategy,Shortcode,Start_Tstamp,Sent,Segments,Revenue,Conversions,Jump Page Clicks,Delivered,Not_Delivered,Optout,Clicks,Cost,Ecpm,Time,Publisher,Campaign,Route,Carrier,Dataset,Message,Responder Template,Keyword,Responder,Router Domain Name,c1,Responded,Response Rate,CTR,Gross Profit,Gross Margin,RPU,Provider,Code_Type,Revenue Source,Ar Day,Sub_Id,Campaign_Id,Roi,Shortcode Name,Total,Jump Page Version,Offer Wall Id,Offer Type,campaign_id,campaign_name,shortcode_DP.SV,Opportunity Cost Send Strategy,Dataset_Agg_30D_eCPM,Opportunity Cost,Offer Vertical,Offer Gap,Vertical Gap,eCPM ratio,Data Vertical,INTERNAL STATUS,Paused Pubs,Status
0,2024-06-10,2024-06-10 12:00,13194 - MFA DIR,13194,I.MFA,461871,I.MFA_461871,672453.0,SS_MFA_I-MFA-7DC_13194_P_10Jun24,910734.0,13194.TF.MFA.454345,MyFam: Check your family's newest relief oppor...,P,1.833269e+10,2024-06-10 15:00:42,NaN,MFA_I.MFA_7DC,61.33,46.0,4635.0,6150.0,NaN,84.000000,1066.0,55.13000,9.972358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toll Free,Toll Free,NaN,NaN,NaN,-6.20,MFATF,NaN,0.0,NaN,Single Offer,NaN,NaN,MFATF_I.MFA,True,16.016711,-37.172770,Loan,0.0,0.0,0.622622,Personal Loan,ACTIVE,NaN,Live
1,2024-06-10,2024-06-10 12:00,13194 - MFA DIR,13194,I.MFA,461871,I.MFA_461871,672453.0,SS_MFA_I-MFA-7DC_13194_P_10Jun24,919578.0,13194.TF.MFA.454341,"MyFam: {{coalesce|first_name|Congrats}}, you r...",P,1.833269e+10,2024-06-10 15:00:42,NaN,MFA_I.MFA_7DC,90.97,132.0,4955.0,6078.0,NaN,84.000000,1381.0,54.52000,14.967094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toll Free,Toll Free,NaN,NaN,NaN,-36.45,MFATF,NaN,0.0,NaN,Single Offer,NaN,NaN,MFATF_I.MFA,True,16.016711,-6.379566,Loan,7.0,7.0,0.934467,Personal Loan,ACTIVE,NaN,Live
2,2024-06-10,2024-06-10 08:00,6322 - BB VAST MBC,6322,PN.SWP,461500,PN.SWP_461500,672423.0,SS_MBC_PN-SWP-30PDC-TMO-OI-SEP23_6322_MI_10Jun24,539488.0,VAST.6322.SC.MBC.452242,"MBC: Today, {{today_mm_dd_yy}}, you received a...",MI,8.083700e+04,2024-06-10 11:00:46,NaN,MBC_PN.SWP_30PDC_TMO_OI_SEP23,19.00,2.0,49.0,2528.0,NaN,3.000000,104.0,18.04264,7.515823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,-15.19,MBC,NaN,0.0,NaN,Single Offer,6322.0,Vast Platinum Card CPA - BriteBox (SMS2),MBC_PN.SWP,True,9.153925,-4.141122,Credit Card,NaN,NaN,0.821049,Sweepstakes,ACTIVE,NaN,Live
3,2024-06-10,2024-06-10 08:00,6322 - BB VAST MBC,6322,WWM.YFA.2,461680,WWM.YFA.2_461680,672457.0,SS_MBC_WWM-YFA-2-1PLD-15DC-TMO_6322_P_10Jun24,461620.0,VAST.6322.SC.MBC-P.1,MBC: Your application has been accepted! Vie...,P,8.083700e+04,2024-06-10 11:01:26,NaN,MBC_WWM.YFA.2_1PLD.15DC_TMO,19.00,2.0,112.0,1437.0,NaN,22.000000,313.0,10.28031,13.221990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,-16.81,MBC,NaN,0.0,NaN,Single Offer,6322.0,Vast Platinum Card CPA - BriteBox (SMS2),MBC_WWM.YFA.2,True,24.921378,-16.812020,Credit Card,8.0,8.0,0.530548,Financial,ACTIVE,NaN,Live
4,2024-06-10,2024-06-10 08:00,6322 - BB VAST MBC,6322,PN.SWP,461500,PN.SWP_461500,672423.0,SS_MBC_PN-SWP-30PDC-TMO-OI-SEP23_6322_MI_10Jun24,461620.0,VAST.6322.SC.MBC-P.1,MBC: Your application has been accepted! Vie...,MI,8.083700e+04,2024-06-10 11:00:46,NaN,MBC_PN.SWP_30PDC_TMO_OI_SEP23,0.00,0.0,92.0,2566.0,NaN,3.000000,189.0,18.31658,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,3.87,MBC,NaN,0.0,NaN,Single Offer,6322.0,Vast Platinum Card CPA - BriteBox (SMS2),MBC_PN.SWP,True,9.153925,-23.488971,Credit Card,NaN,NaN,0.000000,Sweepstakes,ACTIVE,NaN,Live
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [117]:
#calculate Opportunity Cost eCPM
data_offer_vert_OCE['Opportunity Cost eCPM'] = data_offer_vert_OCE['Opportunity Cost'
                                                                  ]*1000/data_offer_vert_OCE['Delivered']

#calculate eCPM
data_offer_vert_OCE['eCPM'] = data_offer_vert_OCE['Revenue']*1000/data_offer_vert_OCE['Delivered']


In [118]:
data_offer_vert_OCE[data_offer_vert_OCE['Opportunity Cost eCPM'] > -3]

,shortcode_DP.SV,Offer Vertical,Revenue,Delivered,Opportunity Cost,Opportunity Cost eCPM,eCPM
1,CSS_AI.CC,Credit Score,100.80000,9973.0,-13.855644,-1.389316,10.107290
2,CSS_AI.CC,Education,262.00000,19796.0,10.216840,0.516106,13.234997
3,CSS_AI.CC,Resources,88.55000,7115.0,-3.838597,-0.539508,12.445538
4,CSS_EDM.247L,Credit Card,4763.00000,367286.0,376.391586,1.024792,12.968096
8,CSS_FSM.YS,Credit Card,1507.00000,162429.0,-268.160206,-1.650938,9.277900
9,CSS_FSM.YS,Credit Score,201.60000,10382.0,101.766084,9.802166,19.418224
13,CSS_I.CC,Credit Card,1346.00000,61214.0,-21.244051,-0.347046,21.988434
15,CSS_I.CC,Education,322.00000,11674.0,9.362572,0.802002,27.582662
17,CSS_NPD.RTO,Credit Card,4121.50000,485099.0,-779.303407,-1.606483,8.496204
19,CSS_NPD.RTO,Education,2098.00000,244736.0,112.312875,0.458914,8.572503


In [119]:
#filter if greater than -1
data_offer_vert_OCE = data_offer_vert_OCE[data_offer_vert_OCE['Opportunity Cost eCPM'] > -3]
data_offer_vert_OCE = data_offer_vert_OCE[data_offer_vert_OCE['eCPM'] > 7]


# Offer Opportunity Cost eCPM

In [120]:
#find data 
offer_OCE = df_dvov_merged.groupby(['Hitpath Offer ID','Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()


In [121]:
#calculate Opportunity Cost eCPM
offer_OCE['Opportunity Cost eCPM'] = offer_OCE['Opportunity Cost']*1000/offer_OCE['Delivered']
offer_OCE['eCPM'] = offer_OCE['Revenue']*1000/offer_OCE['Delivered']


In [122]:
#filter if greater than -1
offer_OCE = offer_OCE[offer_OCE['Opportunity Cost eCPM'] > -5]
offer_OCE = offer_OCE[offer_OCE['eCPM'] > 7]



# Rec

In [123]:
#remove old shortcode_DP.SV
# Combine 'DP.SV' and 'shortcode_DP.SV' to create a unique key
df_dvov_merged['unique_key'] = df_dvov_merged['DP.SV'] + "_" + df_dvov_merged['shortcode_DP.SV']

# Find the latest date for each unique key
latest_dates = df_dvov_merged.groupby('unique_key')['Date'].max()

# Calculate the date 30 days ago from the most recent date in the dataset
date_threshold = df_dvov_merged['Date'].max() - dt.timedelta(days=15)

# Identifiers that have had a drop within the last 30 days
active_keys = latest_dates[latest_dates > date_threshold]

# Filter the original DataFrame to only include rows with these active keys
df_dvov_merged = df_dvov_merged[df_dvov_merged['unique_key'].isin(active_keys.index)]



In [124]:
# Create an empty dataframe for the updated recommendations
recommendations = pd.DataFrame(columns=["shortcode_DP.SV", "PubID", 
                                               "Hitpath Offer ID","Offer Vertical"])

# Iterate over each unique shortcode_DP.SV in df
for shortcode in df_dvov_merged['shortcode_DP.SV'].unique():
    # Filter the dataframe for the specific shortcode
    temp_df = df_dvov_merged[df_dvov_merged['shortcode_DP.SV'] == shortcode]
    
    # Extract the data vertical and PubID
    shortcode_DPSV = temp_df['shortcode_DP.SV'].iloc[0]
    pub_id = temp_df['Affiliate_Id'].iloc[0]
    
    # Iterate over each Hitpath ID in offer_OCE
    for _, row in offer_OCE.iterrows():
        hitpath_id = row['Hitpath Offer ID']
        offer_vertical = row['Offer Vertical']
        
        # Check if this Hitpath ID has been sent by the current shortcode_DP.SV
        if hitpath_id not in temp_df['Hitpath Offer ID'].values:
            # Check if the shortcode_DP.SV's Data Vertical appears in offer_OCE for this Offer Vertical
            potential_matches = data_offer_vert_OCE[(data_offer_vert_OCE['shortcode_DP.SV'] == shortcode_DPSV) & 
                                             (data_offer_vert_OCE['Offer Vertical'] == offer_vertical)]
            if len(potential_matches) > 0:
                # Check if the PubID is not in the Paused Pubs column for this Hitpath ID
                paused_pubs = df_dvov_merged[df_dvov_merged['Hitpath Offer ID'] == hitpath_id]['Paused Pubs'].values
                if len(paused_pubs) > 0 and pub_id not in paused_pubs:
                    recommendations = recommendations.append({
                        "shortcode_DP.SV": shortcode,
                        "PubID": pub_id,
                        "Hitpath Offer ID": hitpath_id,
                        "Offer Vertical": offer_vertical
                    }, ignore_index=True)

recommendations

,shortcode_DP.SV,PubID,Hitpath Offer ID,Offer Vertical
0,MFATF_I.MFA,461871,12089,Education
1,MFATF_I.MFA,461871,12305,Loan
2,MFATF_I.MFA,461871,12306,Loan
3,MFATF_I.MFA,461871,12342,Loan
4,MFATF_I.MFA,461871,12587,Loan
5,MFATF_I.MFA,461871,12610,Loan
6,MFATF_I.MFA,461871,12945,Resources
7,MFATF_I.MFA,461871,13073,Loan
8,MFATF_I.MFA,461871,13074,Loan
9,MFATF_I.MFA,461871,13362,Resources


In [125]:
def filter_recommendations(row):
    if pd.isna(row['Live Pub IDs']):
        return True  # No restrictions, keep the recommendation
    live_pub_ids = row['Live Pub IDs'].split(',')
    current_affiliate_id = str(row['PubID']).strip()
    return current_affiliate_id in live_pub_ids

In [126]:
#checking for paused pubids
recommendations = recommendations.merge(sms_offer[['Hitpath Offer ID', 'Scheduling Name']], on='Hitpath Offer ID', how='left')
recommendations = recommendations.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [127]:
#remove rows if aff id is in pubid restriction
recommendations = recommendations[~recommendations.apply(lambda row: str(row['PubID']) in str(row['Paused Pubs']), axis=1)]


In [128]:
#remove rows if affid doesnt match live pubid
merged_df = recommendations.merge(sms_live_restrictions[['Hitpath Offer ID', 'Live Pub IDs']], on='Hitpath Offer ID', how='left')

# Apply the filter function to the merged dataframe
recommendations = merged_df[merged_df.apply(filter_recommendations, axis=1)]

In [129]:
#drop temp columns
recommendations = recommendations.drop(columns=['Live Pub IDs','PubID','Paused Pubs'], axis = 1)

In [130]:
# mamba_future['Hitpath Offer ID'] = mamba_future['Hitpath Offer ID'].astype(int).astype(str)

#matching hitpath id and shortcode to find matches in rec tab to future scheduledd
matches = recommendations.apply(
    lambda x: any(
        (mamba_future['Hitpath Offer ID'] == x['Hitpath Offer ID']) & 
        (mamba_future['shortcode_DP.SV'] == x['shortcode_DP.SV'])
    ), 
    axis=1
)

# remove matching rows from recommendations tab
recommendations_final = recommendations[~matches]

recommendations_final

,shortcode_DP.SV,Hitpath Offer ID,Offer Vertical,Scheduling Name
0,MFATF_I.MFA,12089,Education,12089-Education-Banner Edge-Degree Snap - SMS ...
4,MFATF_I.MFA,12587,Loan,12587-Loan-EPCVIP-UnitedLoans
5,MFATF_I.MFA,12610,Loan,12610-Loan-AvenueLink-City-Loans - AvenueLink ...
6,MFATF_I.MFA,12945,Resources,12945-Resources-Three Hyphens LLC-LavishGreen ...
7,MFATF_I.MFA,13073,Loan,13073-Loan-Leap Theory-Coastal City Loans - Le...
8,MFATF_I.MFA,13074,Loan,13074-Loan-Leap Theory-Loans Prosper - Leap Th...
9,MFATF_I.MFA,13362,Resources,13362 EXCLUSIVE - Benefit Relief - Housing Rel...
10,MBC_PN.SWP,11793,Resources,11793-Resources-Spark Revenue-ResourceUSA (SMS)
11,MBC_PN.SWP,13348,Sweepstakes,Trendn Daily - $1000 Cashapp - Interest Media ...
13,MBC_PA.PS,11793,Resources,11793-Resources-Spark Revenue-ResourceUSA (SMS)


In [131]:
agg = offer_performance.groupby('Hitpath Offer ID').agg({
    'Overall Revenue': 'sum',
    'Overall Delivered': 'sum'
}).reset_index()

# Calculating eCPM for each aggregated offer ID
agg['eCPM'] = (agg['Overall Revenue'] / agg['Overall Delivered']) * 1000


In [132]:
merged = recommendations_final.merge(agg, left_on='Hitpath Offer ID', right_on='Hitpath Offer ID', how='left')

merged = merged[['shortcode_DP.SV', 'Hitpath Offer ID', 'Offer Vertical', 'Scheduling Name', 'eCPM']]


In [133]:
sorted_data = merged.groupby('shortcode_DP.SV').apply(lambda x: x.sort_values('eCPM', ascending=False)).reset_index(drop=True)


In [134]:
#Limit loan to 2 max
loans = sorted_data[sorted_data['Offer Vertical'] == 'Loan']
other_verticals = sorted_data[sorted_data['Offer Vertical'] != 'Loan']

# Sort the loan data by 'shortcode_DP.SV' and 'eCPM'
sorted_loans = loans.sort_values(by=['shortcode_DP.SV', 'eCPM'], ascending=[True, False])

# Group by 'shortcode_DP.SV' and keep the top 2 loan entries for each group
top_2_loans_per_shortcode = sorted_loans.groupby('shortcode_DP.SV').head(2)

# Concatenate the top 2 loans per shortcode with the other verticals
result = pd.concat([top_2_loans_per_shortcode, other_verticals])
result = result.sort_values(by=['shortcode_DP.SV', 'eCPM'],ascending=[True, False])

In [135]:
#output file
filename = filepath.output_folder+'/Redrop - {}.xlsx'.format(dt.date.today().strftime("%m_%d_%Y"))

#create writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
with pd.ExcelWriter(filename) as writer:
    for dma, df in dma_separated_dfs.items():
        sheet_name = f'{dma} - Redrop'  # dynamic sheet name based on the DMA
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Writing data to different sheets

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    result.to_excel(writer, sheet_name='Offer Recommendations', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    passed_offers.to_excel(writer, sheet_name='Passed Offers - Last 30 days', index=False)

# Close the Pandas Excel writer and output the Excel file.
# writer.close()

In [136]:
toaddr = ['lili@rxmg.com','nathan@rxmg.com', 'prasad@rxmg.com', 'nina@rxmg.com', 'n.ohashi@rxmg.com']
# toaddr = ['nathan@rxmg.com']
today = dt.date.today().strftime("%m_%d_%Y")
subject_line = f"SMS Team Weekly Redrop/Recommendations Report - {today}"
email_body = "Hi Team, here is the Weekly Redrop Report \n\n"

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)
 